In [1]:
from model import get_pretrained_vgg19, train_model, save_model_state
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import TrainDataset, ValidationDataset

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
tf = transforms.Compose([transforms.ToTensor()])

In [3]:
train_ds = TrainDataset(transforms=tf, val_ratio=0.2)
val_ds = ValidationDataset(transforms=tf)

In [4]:
BATCH_SIZE = 64

dl = {'train': DataLoader(train_ds, batch_size=BATCH_SIZE, drop_last=True, shuffle=True), 
    'val': DataLoader(val_ds, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)}

In [6]:
model = get_pretrained_vgg19(18)

In [7]:
best_model_state = train_model(model, dl, 10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


device: cuda


wandb: Currently logged in as: songmingi (use `wandb login --relogin` to force relogin)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
2022-02-28 07:01:30.738927: W tensorflow/stream_executor/platform/

Epoch: 1/10
train loss: 55.83308143130804, train acc: 46.02118682861328
val loss: 23.204698431080786, val acc: 55.32758712768555
Epoch: 2/10
train loss: 18.200648566423837, train acc: 57.84745788574219
val loss: 12.625225597414477, val acc: 59.63793182373047
Epoch: 3/10
train loss: 9.872685154110698, train acc: 60.436439514160156
val loss: 7.865702374228116, val acc: 61.18965530395508
Epoch: 4/10
train loss: 5.3579511932902415, train acc: 62.1016960144043
val loss: 4.282361267969526, val acc: 62.620689392089844
Epoch: 5/10
train loss: 3.3239667286180845, train acc: 62.79661178588867
val loss: 2.4330250954833526, val acc: 63.13793182373047
Epoch: 6/10
train loss: 2.564025605192124, train acc: 63.15254211425781
val loss: 1.7183120114536121, val acc: 63.44827651977539
Epoch: 7/10
train loss: 1.579737320968534, train acc: 63.45338821411133
val loss: 1.326591359387184, val acc: 63.58620834350586
Epoch: 8/10
train loss: 1.0021458061262019, train acc: 63.699153900146484
val loss: 1.0173253412

In [10]:
save_model_state(best_model_state, '/opt/ml/best22281806.pt')

In [11]:
import os
from torch.utils.data import Dataset
import pandas as pd
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from PIL import Image
import torch

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)


test_dir = '/opt/ml/input/data/eval'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, tf)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()
device = torch.device('cuda')
model.to(device)


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
